# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_df = pd.read_csv("cities_data.csv")
cities_df

,City Name,Country,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloudiness,Date
0,sharjah,AE,55.40,25.36,77.00,64,3.36,0,1603231306
1,hermanus,ZA,19.23,-34.42,46.00,91,2.84,0,1603231221
2,mataura,NZ,168.86,-46.19,53.01,69,7.00,36,1603231212
3,shingu,JP,135.98,33.73,61.27,76,9.46,39,1603231222
4,ushuaia,AR,-68.30,-54.80,44.60,68,13.87,75,1603231099
...,...,...,...,...,...,...,...,...,...
549,cruzeiro do sul,BR,-51.99,-29.51,73.00,78,7.61,20,1603231246
550,halifax,CA,-63.57,44.65,62.01,95,9.57,97,1603231261
551,san isidro,PE,-77.05,-12.12,68.00,68,6.93,47,1603231376
552,kholtoson,RU,103.31,50.30,21.42,80,2.35,72,1603231376


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

locations = cities_df[["Latitude", "Longitude"]]

humidity = cities_df["Humidity"]

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrow_city_df = cities_df.loc[(cities_df["Temperature"] >= 70) & (cities_df["Temperature"] <= 80) & \
                               (cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0)].dropna()

narrow_city_df

,City Name,Country,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloudiness,Date
0,sharjah,AE,55.40,25.36,77.00,64,3.36,0,1603231306
29,sur,OM,59.53,22.57,73.15,67,6.15,0,1603231309
125,adre,TD,22.20,13.47,76.84,18,8.37,0,1603231321
185,inhambane,MZ,35.38,-23.86,72.93,90,3.44,0,1603231268
188,poum,NC,164.02,-20.23,74.64,66,4.36,0,1603231059
228,bandarbeyla,SO,50.81,9.49,78.58,73,6.76,0,1603231176
326,garowe,SO,48.48,8.41,78.53,71,6.49,0,1603231348
327,sorbhog,IN,90.87,26.50,71.89,82,3.33,0,1603231348
330,aswan,EG,32.91,24.09,78.03,26,2.66,0,1603231170
342,alice springs,AU,133.88,-23.70,77.00,50,4.70,0,1603231258


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
import json

In [9]:
hotel_df = narrow_city_df
hotel_df["Hotel Name"] = ""

In [10]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result....next up!")
        
hotel_df

Missing field/result....next up!


,City Name,Country,Longitude,Latitude,Temperature,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
0,sharjah,AE,55.40,25.36,77.00,64,3.36,0,1603231306,Swiss Belhotel Sharjah
29,sur,OM,59.53,22.57,73.15,67,6.15,0,1603231309,Sur Plaza Hotel
125,adre,TD,22.20,13.47,76.84,18,8.37,0,1603231321,
185,inhambane,MZ,35.38,-23.86,72.93,90,3.44,0,1603231268,Oceano
188,poum,NC,164.02,-20.23,74.64,66,4.36,0,1603231059,Gîte kajeon
228,bandarbeyla,SO,50.81,9.49,78.58,73,6.76,0,1603231176,REER XOOSHLE GROUP
326,garowe,SO,48.48,8.41,78.53,71,6.49,0,1603231348,Curubo Hotel
327,sorbhog,IN,90.87,26.50,71.89,82,3.33,0,1603231348,Tourist Lodge
330,aswan,EG,32.91,24.09,78.03,26,2.66,0,1603231170,Nuba Nile
342,alice springs,AU,133.88,-23.70,77.00,50,4.70,0,1603231258,Desert Palms Alice Springs


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))